In [1]:
# Setup path and imports
import sys
sys.path.insert(0, '/mnt/home/mlee1/vdm_BIND')

import torch
import numpy as np
import matplotlib.pyplot as plt
from lightning.pytorch import seed_everything

# Set seed for reproducibility
seed_everything(42)

# Check GPU availability
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
if device == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")

/mnt/sw/nix/store/gpkc8q6zjnp3n3h3w9hbmbj6gjbxs85w-python-3.10.10-view/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/mnt/sw/nix/store/6qvrglgqdpwhbw9zv2nh07fpd7a4wq31-py-torchvision-0.15.2/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
Seed set to 42


Using device: cuda
GPU: NVIDIA A100-SXM4-80GB


## 1. Available Methods and Backbones

Let's see what's available in the new API:

In [2]:
from vdm.methods import list_methods, create_method
from vdm.backbones import list_backbones, create_backbone

print("=" * 50)
print("AVAILABLE METHODS")
print("=" * 50)
for method in list_methods():
    print(f"  • {method}")

print("\n" + "=" * 50)
print("AVAILABLE BACKBONES")
print("=" * 50)
for backbone in list_backbones():
    print(f"  • {backbone}")

AVAILABLE METHODS
  • consistency
  • flow
  • vdm

AVAILABLE BACKBONES
  • dit
  • dit-b
  • dit-l
  • dit-s
  • dit-xl
  • fno
  • fno-b
  • fno-l
  • fno-s
  • fno-xl
  • unet
  • unet-b
  • unet-l
  • unet-s


## 2. Creating Backbones

All backbones follow a unified interface:
```python
output = backbone(x_t, t, conditioning, param_conditioning)
```

Where:
- `x_t`: (B, C, H, W) - Noisy input
- `t`: (B,) - Time in [0, 1]
- `conditioning`: (B, C_cond, H, W) - Spatial conditioning (DM + large-scale)
- `param_conditioning`: (B, N_params) - Cosmological parameters

In [3]:
# Create different backbone architectures

# Common parameters
backbone_params = {
    'input_channels': 3,
    'output_channels': 3,
    'conditioning_channels': 1,
    'large_scale_channels': 3,
    'param_dim': 6,  # Cosmological parameters
    'img_size': 128,
}

# Create a small UNet backbone
unet_backbone = create_backbone('unet-s', **backbone_params)
print(f"UNet-S parameters: {sum(p.numel() for p in unet_backbone.parameters()):,}")

# Create a small DiT backbone  
dit_backbone = create_backbone('dit-s', **backbone_params)
print(f"DiT-S parameters: {sum(p.numel() for p in dit_backbone.parameters()):,}")

# Create a small FNO backbone
fno_backbone = create_backbone('fno-s', **backbone_params)
print(f"FNO-S parameters: {sum(p.numel() for p in fno_backbone.parameters()):,}")

✓ Legacy Fourier features enabled (backward compatibility):
  - Exponential frequencies: 2^(-2:1) * 2π
  - Applied to first channel of concatenated input
  - 8 Fourier features
  - Total input channels: 15
⚙️  ParamEmbedding: Unconditional mode (n_params=0)
✓ Parameter conditioning enabled:
  - Time embedding dim: 256
  - Param embedding dim: 256
  - Total condition dim: 512
✓ Channel progression (encoder): [64, 128, 256]
✓ Final decoder channels: 64 (GroupNorm groups: 8)
✓ Parameter predictor added (predicts 0 params)
UNet-S parameters: 10,139,781


/mnt/home/mlee1/venvs/torch3/lib/python3.10/site-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")



INITIALIZED DiT MODEL
  Image size: 128x128
  Patch size: 4
  Num patches: 1024
  Hidden size: 384
  Depth: 12
  Heads: 6
  MLP ratio: 4.0
  Parameters: 6 (0=unconditional)
  Conditioning: 1 + 3 large-scale
  Total params: 32,674,224

DiT-S parameters: 32,674,224

INITIALIZED FNO2d
  Input channels: 3
  Output channels: 3
  Conditioning channels: 4
  Hidden channels: 32
  FNO layers: 4
  Param conditioning: True (6 params)
  Total parameters: 2,147,267

FNO-S parameters: 2,147,267


In [4]:
# Test backbone interface
batch_size = 2
x_t = torch.randn(batch_size, 3, 128, 128)  # Noisy input
t = torch.rand(batch_size)  # Time in [0, 1]
conditioning = torch.randn(batch_size, 4, 128, 128)  # DM + 3 large-scale
param_cond = torch.randn(batch_size, 6)  # Cosmological params

# Test UNet
with torch.no_grad():
    out_unet = unet_backbone(x_t, t, conditioning, param_cond)
    print(f"UNet output shape: {out_unet.shape}")

# Test DiT
with torch.no_grad():
    out_dit = dit_backbone(x_t, t, conditioning, param_cond)
    print(f"DiT output shape: {out_dit.shape}")

# Test FNO
with torch.no_grad():
    out_fno = fno_backbone(x_t, t, conditioning, param_cond)
    print(f"FNO output shape: {out_fno.shape}")

UNet output shape: torch.Size([2, 3, 128, 128])
DiT output shape: torch.Size([2, 3, 128, 128])
FNO output shape: torch.Size([2, 3, 128, 128])


## 3. Creating Methods

Methods wrap backbones and provide:
- `compute_loss(x, conditioning, params)` - Training loss
- `sample(conditioning, n_samples, n_steps)` - Generation
- `draw_samples(x, conditioning, params, ...)` - BIND-compatible interface

In [5]:
# Create methods with different backbones

# VDM with UNet (classic setup)
vdm_unet = create_method(
    'vdm',
    backbone_type='unet-s',
    img_size=128,
    param_dim=6,
    conditioning_channels=1,
    large_scale_channels=3,
    learning_rate=1e-4,
)
print(f"VDM+UNet parameters: {sum(p.numel() for p in vdm_unet.parameters()):,}")

# Flow Matching with DiT
flow_dit = create_method(
    'flow',
    backbone_type='dit-s',
    img_size=128,
    param_dim=6,
    conditioning_channels=1,
    large_scale_channels=3,
    n_sampling_steps=50,
)
print(f"Flow+DiT parameters: {sum(p.numel() for p in flow_dit.parameters()):,}")

# Consistency with FNO (fast sampling!)
cons_fno = create_method(
    'consistency',
    backbone_type='fno-s',
    img_size=128,
    param_dim=6,
    conditioning_channels=1,
    large_scale_channels=3,
    n_sampling_steps=1,  # Single step!
)
print(f"Consistency+FNO parameters: {sum(p.numel() for p in cons_fno.parameters()):,}")

✓ Legacy Fourier features enabled (backward compatibility):
  - Exponential frequencies: 2^(-2:1) * 2π
  - Applied to first channel of concatenated input
  - 8 Fourier features
  - Total input channels: 15
⚙️  ParamEmbedding: Unconditional mode (n_params=0)
✓ Parameter conditioning enabled:
  - Time embedding dim: 256
  - Param embedding dim: 256
  - Total condition dim: 512
✓ Channel progression (encoder): [64, 128, 256]
✓ Final decoder channels: 64 (GroupNorm groups: 8)
✓ Parameter predictor added (predicts 0 params)

VDM METHOD INITIALIZED
  Backbone: UNetBackbone
  Gamma range: [-13.3, 5.0]
  Noise schedule: fixed_linear
  Sampling steps: 250
  Channel weights: (1.0, 1.0, 1.0)
  Focal loss: False

VDM+UNet parameters: 10,139,781

INITIALIZED DiT MODEL
  Image size: 128x128
  Patch size: 4
  Num patches: 1024
  Hidden size: 384
  Depth: 12
  Heads: 6
  MLP ratio: 4.0
  Parameters: 6 (0=unconditional)
  Conditioning: 1 + 3 large-scale
  Total params: 32,674,224


FLOW MATCHING METHOD

## 4. Loading Data

Let's load a small subset of training data (1000 samples as requested):

In [10]:
from vdm.astro_dataset import get_astro_data

# Load training data
data_root = '/mnt/home/mlee1/ceph/train_data_rotated2_128_cpu/train/'

# Use a small batch size for this tutorial
datamodule = get_astro_data(
    dataset='IllustrisTNG',
    data_root=data_root,
    batch_size=32,
    num_workers=4,
    stage='fit',
    limit_train_samples=1000)#  Limit to 1000 samples for quick testing)

datamodule.setup('fit')
print(f"Training samples: {len(datamodule.train_dataset)}")
print(f"Validation samples: {len(datamodule.val_dataset)}")

✓ Loaded DM stats: mean=9.915499, std=0.467967
✓ Loaded Gas stats: mean=9.177344, std=0.410510
✓ Loading stellar Z-score stats from: /mnt/home/mlee1/vdm_BIND/data/stellar_normalization_stats.npz
  mean=6.995926, std=1.100350

📊 NORMALIZATION CONFIG (3-channel mode):
  DM input: mean=9.9777, std=0.4139
  DM target: mean=9.9155, std=0.4680
  Gas target: mean=9.1773, std=0.4105
  Stars target: mean=6.9959, std=1.1004
✓ Loaded DM stats: mean=9.915499, std=0.467967
✓ Loaded Gas stats: mean=9.177344, std=0.410510
✓ Loading stellar Z-score stats from: /mnt/home/mlee1/vdm_BIND/data/stellar_normalization_stats.npz
  mean=6.995926, std=1.100350

📊 NORMALIZATION CONFIG (3-channel mode):
  DM input: mean=9.9777, std=0.4139
  DM target: mean=9.9155, std=0.4680
  Gas target: mean=9.1773, std=0.4105
  Stars target: mean=6.9959, std=1.1004
📂 Loading file list from cache: /mnt/home/mlee1/ceph/train_data_rotated2_128_cpu/train/file_list_cache.txt
   Loaded 408860 files in 0.48s
Found 408860 total files


AttributeError: 'AstroDataModule' object has no attribute 'train_dataset'

In [11]:
# Inspect a batch
train_loader = datamodule.train_dataloader()
batch = next(iter(train_loader))

# Unpack batch
condition, target, params = batch['condition'], batch['target'], batch['params']

print(f"Condition shape: {condition.shape}")  # (B, 1+3, H, W) = DM + large-scale
print(f"Target shape: {target.shape}")  # (B, 3, H, W) = [DM_hydro, Gas, Stars]
print(f"Params shape: {params.shape}")  # (B, N_params)

TypeError: list indices must be integers or slices, not str

In [ ]:
# Visualize a sample
idx = 0

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# Condition channels
axes[0, 0].imshow(condition[idx, 0].numpy(), cmap='viridis')
axes[0, 0].set_title('DM Condition')
axes[0, 1].imshow(condition[idx, 1].numpy(), cmap='viridis')
axes[0, 1].set_title('Large-scale 1')
axes[0, 2].imshow(condition[idx, 2].numpy(), cmap='viridis')
axes[0, 2].set_title('Large-scale 2')
axes[0, 3].imshow(condition[idx, 3].numpy(), cmap='viridis')
axes[0, 3].set_title('Large-scale 3')

# Target channels
axes[1, 0].imshow(target[idx, 0].numpy(), cmap='inferno')
axes[1, 0].set_title('DM Hydro (target)')
axes[1, 1].imshow(target[idx, 1].numpy(), cmap='inferno')
axes[1, 1].set_title('Gas (target)')
axes[1, 2].imshow(target[idx, 2].numpy(), cmap='inferno')
axes[1, 2].set_title('Stars (target)')
axes[1, 3].axis('off')

for ax in axes.flat:
    ax.axis('off')
    
plt.tight_layout()
plt.show()

## 5. Quick Training Demo

Let's do a quick training run with VDM + UNet-S on our small dataset:

In [ ]:
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger

# Create a fresh VDM model for training
model = create_method(
    'vdm',
    backbone_type='unet-s',
    img_size=128,
    param_dim=params.shape[1],
    conditioning_channels=1,
    large_scale_channels=3,
    learning_rate=1e-4,
)

print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# Quick training (just a few epochs for demo)
logger = TensorBoardLogger(
    save_dir='/mnt/home/mlee1/ceph/tb_logs_tutorial',
    name='vdm_unet_tutorial',
    version=0
)

trainer = Trainer(
    max_epochs=5,  # Just a few epochs for demo
    accelerator='gpu' if device == 'cuda' else 'cpu',
    devices=1,
    logger=logger,
    enable_checkpointing=True,
    log_every_n_steps=10,
    limit_train_batches=0.5,  # Use only 50% of training data per epoch
    limit_val_batches=0.2,
)

print("Starting training...")
trainer.fit(model, datamodule)

## 6. Sampling / Generation

Now let's generate samples using our trained model:

In [ ]:
# Move model to GPU and set to eval mode
model = model.to(device).eval()

# Get a batch for conditioning
val_loader = datamodule.val_dataloader()
val_batch = next(iter(val_loader))

condition = val_batch['condition'].to(device)
target = val_batch['target'].to(device)
params = val_batch['params'].to(device)

print(f"Condition: {condition.shape}")
print(f"Target: {target.shape}")
print(f"Params: {params.shape}")

In [ ]:
# Generate samples using the draw_samples interface (BIND compatible)
n_samples = 4  # Generate 4 samples

with torch.no_grad():
    # Use just the first condition for demo
    cond_single = condition[:1]  # (1, 4, 128, 128)
    param_single = params[:1]  # (1, N_params)
    
    # Generate multiple realizations
    samples = model.draw_samples(
        x=cond_single.expand(n_samples, -1, -1, -1),
        conditioning=cond_single.expand(n_samples, -1, -1, -1),
        conditional_params=param_single.expand(n_samples, -1),
        n_sampling_steps=50,  # Use 50 steps for VDM
    )
    
print(f"Generated samples shape: {samples.shape}")

In [ ]:
# Visualize generated samples vs ground truth
fig, axes = plt.subplots(n_samples + 1, 3, figsize=(12, 4*(n_samples+1)))

channel_names = ['DM Hydro', 'Gas', 'Stars']

# Ground truth (first row)
for c in range(3):
    im = axes[0, c].imshow(target[0, c].cpu().numpy(), cmap='inferno')
    axes[0, c].set_title(f'Ground Truth - {channel_names[c]}')
    axes[0, c].axis('off')
    plt.colorbar(im, ax=axes[0, c], fraction=0.046)

# Generated samples
for i in range(n_samples):
    for c in range(3):
        im = axes[i+1, c].imshow(samples[i, c].cpu().numpy(), cmap='inferno')
        axes[i+1, c].set_title(f'Sample {i+1} - {channel_names[c]}')
        axes[i+1, c].axis('off')
        plt.colorbar(im, ax=axes[i+1, c], fraction=0.046)

plt.tight_layout()
plt.show()

## 7. Config Factory

The new config factory makes it easy to generate training configs:

In [ ]:
from configs.config_factory import ConfigFactory

# List available presets
print("Methods:", ConfigFactory.list_methods())
print("Backbones:", ConfigFactory.list_backbones())

In [ ]:
# Create a config for Flow + DiT
config = ConfigFactory.create(
    method='flow',
    backbone='dit-s',
    model_name='flow_dit_tutorial',
    batch_size=64,
    max_epochs=100,
)

# Print some key settings
print("Generated config:")
for key in ['method', 'backbone_type', 'model_name', 'batch_size', 'learning_rate', 'n_sampling_steps']:
    print(f"  {key}: {config.get(key, 'N/A')}")

In [ ]:
# Save config to file
ConfigFactory.save(config, '/mnt/home/mlee1/vdm_BIND/configs/tutorial_flow_dit.ini')
print("Config saved to: configs/tutorial_flow_dit.ini")

## 8. Training from Command Line

You can train using `train_zoo.py` from the command line:

```bash
# With auto-generated config
python train_zoo.py --method vdm --backbone unet-s

# With saved config
python train_zoo.py --config configs/tutorial_flow_dit.ini

# List available options
python train_zoo.py --list

# Generate config only (don't train)
python train_zoo.py --method consistency --backbone fno-s --generate-config
```

## 9. Comparison: Method vs Backbone

Let's create a quick comparison showing how different methods work with the same backbone:

In [ ]:
# Compare loss computation across methods
methods_to_compare = ['vdm', 'flow', 'consistency']
backbone_type = 'unet-s'

# Create models
models = {}
for method in methods_to_compare:
    models[method] = create_method(
        method,
        backbone_type=backbone_type,
        img_size=128,
        param_dim=6,
        conditioning_channels=1,
        large_scale_channels=3,
    ).to(device)
    print(f"{method.upper()}: {sum(p.numel() for p in models[method].parameters()):,} params")

In [ ]:
# Compute loss for each method on same batch
print("\nLoss computation comparison:")
print("=" * 50)

for method_name, method_model in models.items():
    method_model.train()
    
    # Compute loss
    loss_dict = method_model.compute_loss(
        x=target[:8].to(device),  # Use small batch
        conditioning=condition[:8].to(device),
        params=params[:8].to(device),
    )
    
    loss = loss_dict['loss'].item()
    print(f"{method_name.upper():12s}: loss = {loss:.4f}")

## Summary

The new Generative Zoo API provides:

1. **Unified Backbone Interface** - All architectures (UNet, DiT, FNO) follow the same API
2. **Modular Methods** - VDM, Flow Matching, and Consistency training paradigms
3. **Easy Configuration** - ConfigFactory for generating training configs
4. **BIND Compatibility** - `draw_samples()` interface works with existing BIND pipeline
5. **Systematic Comparison** - Mix and match methods with backbones

### Quick Reference

| Method | Sampling Steps | Best For |
|--------|----------------|----------|
| VDM | 250-1000 | Quality |
| Flow | 50-100 | Balance |
| Consistency | 1-5 | Speed |

| Backbone | Parameters | Best For |
|----------|------------|----------|
| UNet | ~30M | General purpose |
| DiT | ~50M | Scalability |
| FNO | ~10M | Multi-scale physics |